# ETL Pipeline

This notebook creates an ETL Pipeline that transfers data from files in two local directories `song_data` and `log_data` into tables in Postgres using python and SQL. 

The following files and libraries are imported: `psycopg2` Python sql library, `create_tables.py` that creates the different tables for the sparkify database and `etl.py` that processes the files and inserts into the databases.

In [1]:
import psycopg2
from create_tables import *
from etl import *

In [2]:
%load_ext sql
%sql postgresql:///sparkify_db

### Connect to the sparkify database

In [3]:
try:
    conn = psycopg2.connect("dbname=sparkify_db user=china password=test")
except psycopg2.Error as e:
    print(e)

cur = conn.cursor()
conn.set_session(autocommit=True)

### Create Tables 

Check if the tables already exist in the sparkify database, and drop them if they exist.

In [4]:
drop_tables(cur)

The function below creates the songs table. This table will store the songs in the sparkify music database. The table has the following columns: *song_id*, *title*, *artist_id*, *year*, *duration*.

In [5]:
create_songs_table(cur)

In [6]:
%%sql
SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'songs';

 * postgresql:///sparkify_db
5 rows affected.


column_name,data_type
year,integer
duration,double precision
song_id,character varying
title,character varying
artist_id,character varying


Next, create users table with the following columns *user_id*, *first_name*, *last_name*, *gender*, *level*. Then display the table colummns and data type for the users table to verify it's been created properly. 

In [7]:
create_users_table(cur)


In [8]:
%%sql
SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'songs';

 * postgresql:///sparkify_db
5 rows affected.


column_name,data_type
year,integer
duration,double precision
song_id,character varying
title,character varying
artist_id,character varying


Follow the same process for creating and displaying the remaining tables artists, time, and songplays. 

In [9]:
create_artists_table(cur)

In [10]:
%%sql
SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'artists';

 * postgresql:///sparkify_db
5 rows affected.


column_name,data_type
latitude,double precision
longitude,double precision
artist_id,character varying
name,character varying
location,character varying


In [11]:
create_time_table(cur)

In [12]:
%%sql
SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'time';

 * postgresql:///sparkify_db
7 rows affected.


column_name,data_type
week,integer
hour,integer
day,integer
month,integer
year,integer
start_time,character varying
weekday,character varying


In [13]:
create_songplays(cur)

In [14]:
%%sql
SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'songplays';

 * postgresql:///sparkify_db
9 rows affected.


column_name,data_type
start_time,time without time zone
session_id,integer
user_id,integer
songplay_id,integer
location,character varying
user_agent,character varying
level,text
song_id,character varying
artist_id,character varying


### Read and process files in song_data directory

Walk through the `song_data` directory and extract all the JSON files in the directory. Open and read all the JSON files, process the data and load the processed data into songs and artists tables.

In [15]:
#Read song_data_files
song_data_files = get_listof_files("song_data")

In [16]:
#Insert into songs and artists tables
read_song_files(song_data_files, cur)

In [17]:
%%sql
SELECT * FROM songs LIMIT 5;

 * postgresql:///sparkify_db
5 rows affected.


song_id,title,artist_id,year,duration
SONHOTT12A8C13493C,Something Girls,AR7G5I41187FB4CE6C,1982,233.40363
SOIAZJW12AB01853F1,Pink World,AR8ZCNI1187B9A069B,1984,269.81832
SOFSOCN12A8C143F5D,Face the Ashes,ARXR32B1187FB57099,2007,209.60608
SOHKNRJ12A6701D1F8,Drop of Rain,AR10USD1187B99F3F1,0,189.57016
SOQHXMF12AB0182363,Young Boy Blues,ARGSJW91187B9B1D6B,0,218.77506


In [18]:
%%sql
SELECT * FROM artists LIMIT 5;

 * postgresql:///sparkify_db
5 rows affected.


artist_id,name,location,latitude,longitude
AR7G5I41187FB4CE6C,Adam Ant,"London, England",0.0,0.0
AR8ZCNI1187B9A069B,Planet P Project,,0.0,0.0
ARXR32B1187FB57099,Gob,,0.0,0.0
AR10USD1187B99F3F1,Tweeterfriendly Music,"Burlington, Ontario, Canada",0.0,0.0
ARGSJW91187B9B1D6B,JennyAnyKind,North Carolina,35.21962,-80.01955


The first 5 rows from both the songs and artists table are displayed above.

### Read and process files in log_data directory

Walk through the `log_data` directory and extract all the JSON files in the directory. Open and read all the JSON files, process the data and load the processed data into time, users and songplays tables. Use pandas to transform the timestamp data into *hour*, *day*, *week*, *month*, *year* and *weekday* before loading the data into the time table. Also join the songs and artists tables created earlier to get the *song_id* and *artist_id* to load into the respective columns for the songplays table. 

In [19]:
#Read log data files
log_data_files = get_listof_files("log_data")

In [20]:
#Insert intoo time, songplays and users tables
read_log_files(log_data_files, cur)

In [21]:
%%sql
SELECT * FROM users LIMIT 5;

 * postgresql:///sparkify_db
5 rows affected.


user_id,first_name,last_name,gender,level
69,Anabelle,Simpson,F,free
66,Kevin,Arellano,M,free
40,Tucker,Garrison,M,free
43,Jahiem,Miles,M,free
8,Kaylee,Summers,F,free


In [22]:
%%sql
SELECT * FROM songplays LIMIT 5;

 * postgresql:///sparkify_db
5 rows affected.


songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
1,02:33:56.796000,69,free,None,None,455,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"""
2,00:11:00.796000,66,free,None,None,815,"Harrisburg-Carlisle, PA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"""
3,03:22:04.796000,99,free,None,None,573,"Salt Lake City, UT",Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:31.0) Gecko/20100101 Firefox/31.0
4,02:54:52.796000,43,free,None,None,42,"San Antonio-New Braunfels, TX","""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"""
5,21:05:52.796000,8,free,None,None,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"""


In [23]:
%%sql
SELECT * FROM time LIMIT 5;

 * postgresql:///sparkify_db
5 rows affected.


start_time,hour,day,week,month,year,weekday
2018-11-11 02:33:56.796000,2,11,45,11,2018,Sunday
2018-11-23 00:11:00.796000,0,23,47,11,2018,Friday
2018-11-18 03:22:04.796000,3,18,46,11,2018,Sunday
2018-11-04 02:54:52.796000,2,4,44,11,2018,Sunday
2018-11-01 21:05:52.796000,21,1,44,11,2018,Thursday


The first 5 rows from users, songplays, and time are dispalyed above.

### Close connection to sparkify database

In [24]:
#Close database connection
cur.close()
conn.close()